# Uploading Data to Big Query

This notebook downloads data from the ZTF Public Alerts Archive and uploads it into big query.

In [ ]:
import os
import shutil

import numpy as np
from broker import ztf_archive as ztfa
from broker.alert_acquisition.ztf import map_alert_list_to_schema
from broker.data_upload import batch_ingest
from tqdm import tqdm


We start by retrieving a list of available ZTF releases.

In [ ]:
release_list = ztfa.get_remote_release_list()
release_list.pop()  # Remove checksums file
print(release_list[:5])  # Print the first 5 for demonstration


Next we get the year, month, and date for each release. Here we only keep releases from December 2018.

In [ ]:
alert_dates = []
for release in release_list:
    date_str = release.strip('ztf_public_').strip('.tar.gz')
    date_tuple = (int(date_str[:4]), int(date_str[4:6]), int(date_str[6:]))
    if date_tuple[0] == 2018 and date_tuple[1] == 12:
        alert_dates.append(date_tuple)

print(f'Number of releases: {len(alert_dates)}')


In [ ]:
alert_dates

Since we will be uploading data iterativly, we expect a few upload events to time out or fail for other seemingly "random" reasons. We thus define an `upload` function to help handel these problems.

In [ ]:
def upload(data, data_set, table_name, max_tries=5):
    """Batch upload a Pandas DataFrame into a BigQuery table

    If the upload fails, retry until success or until 
    max_tries is reached.

    Args:
        data (DataFrame): Data to upload to table
        data_set   (str): The name of the data set
        table      (str): The name of the table
        max_tries  (int): Maximum number of tries until error
    """

    i = 0
    while True:
        if i >= max_tries:
            break

        try:
            batch_ingest(data, data_set, table_name)
            return
        
        except KeyboardInterrupt:
            raise
        
        except Exception as e:
            print(f'Error uploading to table {table_name}: {str(e)}')
            print('Trying again...')
            i += 1
            continue
            
        else:
            break
    
    raise RuntimeError('Could not upload data.')


Finally we download data from ZTF and upload it into BigQuery.

In [ ]:
for date in alert_dates:
    
    try:
        ztfa.download_data_date(*date)
    
    except KeyboardInterrupt:
        raise
    
    except:
        continue
    
    print('Uploading to Big Query.')
    total_iter = np.ceil(len(ztfa.get_local_alert_list()) / 800)
    iter_data = ztfa.iter_alerts(800)
    for data in tqdm(iter_data, total=total_iter):
        alert_df, candidate_df = map_alert_list_to_schema(data)
        upload(alert_df, 'ztf_alerts', 'alert')
        upload(candidate_df, 'ztf_alerts', 'candidate')
    
    print('Deleting Local Data.')
    shutil.rmtree(ztfa._download_data.DATA_DIR)
    os.mkdir(ztfa._download_data.DATA_DIR)


After adding data into BigQuery, it's a good idea to check for duplicate data. To select only the unique columns, use the following SQL statement in BigQuery.

```
SELECT * EXCEPT(row_number)
FROM (
  SELECT
      *,
      ROW_NUMBER()
          OVER (PARTITION BY candID)
          row_number
  FROM ztf_alerts.alert
)
WHERE row_number = 1;
```